In [ ]:
!pip install simpletransformers

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Bert_NER/ner_dataset.csv" , encoding="latin1")
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [ ]:
df.fillna(method='ffill',inplace=True)

In [ ]:
df.head(10)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O
5,Sentence: 1,through,IN,O
6,Sentence: 1,London,NNP,B-geo
7,Sentence: 1,to,TO,O
8,Sentence: 1,protest,VB,O
9,Sentence: 1,the,DT,O


In [ ]:
df.rename(columns={"Sentence #":"sentence_id"} , inplace=True)

In [ ]:
df.rename(columns={"Word":"words","Tag":"labels"}, inplace =True)

In [ ]:
from sklearn.preprocessing import LabelEncoder

df['sentence_id'] = LabelEncoder().fit_transform(df.sentence_id).copy()
df.head()

,sentence_id,words,POS,labels
0,0,Thousands,NNS,O
1,0,of,IN,O
2,0,demonstrators,NNS,O
3,0,have,VBP,O
4,0,marched,VBN,O


In [ ]:
df.tail()

,sentence_id,words,POS,labels
1048570,42177,they,PRP,O
1048571,42177,responded,VBD,O
1048572,42177,to,TO,O
1048573,42177,the,DT,O
1048574,42177,attack,NN,O


In [ ]:
df["labels"] = df["labels"].str.upper()

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df,random_state=0,test_size=0.3)

In [ ]:
print(train.shape)
print(test.shape)

(734002, 4)
(314573, 4)


In [ ]:
from simpletransformers.ner import NERModel, NERArgs

In [ ]:
ner = NERArgs()
ner.train_batch_size=32
ner.eval_batch_size=32
ner.num_train_epochs=4
ner.learning_rate=1e-4
ner.overwrite_output_dir=True

In [ ]:
label = df["labels"].unique().tolist()
label

['O',
 'B-GEO',
 'B-GPE',
 'B-PER',
 'I-GEO',
 'B-ORG',
 'I-ORG',
 'B-TIM',
 'B-ART',
 'I-ART',
 'I-PER',
 'I-GPE',
 'I-TIM',
 'B-NAT',
 'B-EVE',
 'I-EVE',
 'I-NAT']

In [ ]:
train.drop(['POS'] , axis=1,inplace=  True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
test.drop(['POS'] , axis=1,inplace=  True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
model = NERModel('bert' ,  'bert-base-cased' , labels=label ,  args = ner)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [ ]:
from sklearn.metrics import accuracy_score

model.train_model(train_data=train , eval_data=test ,acc = accuracy_score)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/1499 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/simpletransformers/ner/ner_model.py:769: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  model.parameters(), args.max_grad_norm
/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running Epoch 1 of 4:   0%|          | 0/1499 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/1499 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/1499 [00:00<?, ?it/s]

(5996, 0.12897214120511136)

In [ ]:
result, model_outputs, preds_list = model.eval_model(test)

  0%|          | 0/5 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1490 [00:00<?, ?it/s]

In [ ]:
result

{'eval_loss': 0.21501098414945521,
 'f1_score': 0.7925843139833015,
 'precision': 0.811432930249254,
 'recall': 0.7745914805217056}

In [ ]:
%cd /content/drive/MyDrive/Bert_NER

/content/drive/MyDrive/Bert_NER


In [ ]:
prediction, model_output = model.predict(["What is the new name of Bangalore"])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
prediction

[[{'What': 'O'},
  {'is': 'O'},
  {'the': 'O'},
  {'new': 'O'},
  {'name': 'O'},
  {'of': 'O'},
  {'Bangalore': 'B-GEO'}]]

In [ ]:
import pickle
file =open('net.obj', 'wb')
pickle.dump(model, file)

In [ ]:
file.close()

In [ ]:
file = open('net.obj', 'rb')

# dump information to that file
data = pickle.load(file)


In [ ]:
p, model_output = data.predict(["What is the new name of London"])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
p

[[{'What': 'O'},
  {'is': 'O'},
  {'the': 'O'},
  {'new': 'O'},
  {'name': 'O'},
  {'of': 'O'},
  {'London': 'B-GEO'}]]